## Задание 1

Какова вероятность отправиться на прогулку, если идёт дождь, при наличии следующих наблюдений?

```
data = [
        ('солнечно', True),
        ('снег', False),
        ('облачно', False),
        ('дождь', False),
        ('солнечно', True),
        ('снег', False),
        ('облачно', True),
        ('снег', False),
        ('солнечно', False),
        ('облачно', True),
        ('снег', True),
        ('солнечно', True),
        ('дождь', False),
        ('дождь', True),
        ('облачно', True),
]

***Решение:***

1) Рассмотрим следующие гипотезы:

    1. $H_1 =$ будет *солнечнно*;

    2. $H_2 =$ будет *снег*;

    3. $H_3 =$ будет *облачно*;

    4. $H_4 =$ будет *дождь*.

    Данные гипотезы образуют полную группу.

2) Вероятность возникновения гипотез (априорная вероятность гипотез $H_i, i = \overline{1, 4}$):

    1. $P(H_1) = \frac{4}{15}$;

    2. $P(H_2) = \frac{4}{15}$;

    3. $P(H_3) = \frac{4}{15}$;
    
    4. $P(H_4) = \frac{3}{15}$.

    Проверка: $\sum\limits_{i=1}^4 P(H_i) = 1$

3) Пусть зависимое событие $A$ – отправиться на прогулку. Оно может наступить, если наступит одна из гипотез $H_i, i = \overline{1, 4}$. Вероятность пойти гулять при наступлении гипотезы $H_i$:

    1. при $i = 1: P(A|H_1) = \frac{3}{4}$;

    2. при $i = 2: P(A|H_2) = \frac{1}{4}$;

    3. при $i = 3: P(A|H_3) = \frac{3}{4}$;

    4. при $i = 4: P(A|H_4) = \frac{1}{3}$.

4) Значит, полная вероятность события $A$ равна: $P(A) = \sum\limits_{i=1}^4 P(H_i) \cdot P(A|H_i) = \frac{4}{15} \cdot \frac{3}{4} + \frac{4}{15} \cdot \frac{1}{4} + \frac{4}{15} \cdot \frac{3}{4} + \frac{3}{15} \cdot \frac{1}{3} = \frac{8}{15}$.

5) Получается, по формулам Байеса, вероятность дождя при походе на прогулку: $P(H_4|A) = \frac{P(H_4) \cdot P(A|H_4)}{P(A)} = \frac{1}{8}$.


## Задание 2

- загрузите датасет для регрессии, проведите предобработку, выделите целевой признак и предикторы, разбейте данные на обучающую и тестовую выборку;
- Масштабируйте числовые признаки
- В случае дисбаланса проведите балансировку
- решите задачу классификации на ваших данных с использованием рассмотренных моделей sklearn: knn, NB, Logistic Regression, SVM. Не забудьте подобрать гиперпараметры.
- вычислите значения метрик Accuracy, Precision, Recall, F1, ROC AUC score

In [59]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [22]:
table = pd.read_csv("../data/neo_task.csv")
table.head(5)

,id,name,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,absolute_magnitude,hazardous
0,3561024.0,(2011 GZ2),0.016016,0.035813,56014.078517,1.024333e+06,26.1,False
1,54016766.0,(2020 HT6),0.030518,0.068240,7864.348060,3.268186e+07,24.7,False
2,3746620.0,(2016 ED156),0.055533,0.124177,55257.544508,6.538636e+07,23.4,False
3,3633054.0,(2013 FD8),0.019256,0.043057,41531.404722,1.260796e+07,25.7,False
4,3742124.0,(2016 CW31),0.139494,0.311918,67639.394481,7.130590e+07,21.4,False


In [23]:
table.drop(columns=['id', 'name'], axis=1, inplace=True)
table.head(5)

,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,absolute_magnitude,hazardous
0,0.016016,0.035813,56014.078517,1.024333e+06,26.1,False
1,0.030518,0.068240,7864.348060,3.268186e+07,24.7,False
2,0.055533,0.124177,55257.544508,6.538636e+07,23.4,False
3,0.019256,0.043057,41531.404722,1.260796e+07,25.7,False
4,0.139494,0.311918,67639.394481,7.130590e+07,21.4,False


In [24]:
null_columns = []
for i in table.columns:
    if len(table[table[i].isnull()]) > 0:
        null_columns.append(i)
print(null_columns)

['est_diameter_max', 'relative_velocity', 'absolute_magnitude']


In [25]:
def fill_empty_cell(column_name, df):
    if df.dtypes[column_name] == "float64":
        df[column_name] = df[column_name].fillna(df[column_name].mean())
    elif df.dtypes[column_name] == "int64":
        df[column_name] = df[column_name].fillna(df[column_name].median())
    else:
        df[column_name] = df[column_name].fillna(df[column_name].mode())


for name in null_columns:
    fill_empty_cell(name, table)

In [26]:
for i in table.columns:
    if len(table[table[i].isnull()]) > 0:
        print(f"В столбце {i} есть пустые элементы")

In [27]:
table.to_csv('../data/neo_task_compleated.csv', index=False) 

In [41]:
table.groupby(['hazardous'])['hazardous'].count()
# дисбаланс классов явно присутствует!!!

hazardous
False    81996
True      8840
Name: hazardous, dtype: int64

In [28]:
X = np.array(table.drop(columns=['hazardous'], axis=1))
y = np.array(table['hazardous'])

In [29]:
# масштабирование столбиков (очень большой разлет значений)
means, stds = X.mean(axis=0), X.std(axis=0)
X = (X - means) / stds
print(X[:3])
X

[[-0.37324117 -0.37376222  0.31519076 -1.61248863  0.8925783 ]
 [-0.32466019 -0.32510641 -1.59424649 -0.19616596  0.40698129]
 [-0.24085864 -0.24117577  0.28518947  1.26699715 -0.04393023]]


In [46]:
# решение проблемы дисбаланса классов
smt = SMOTE()
X, y = smt.fit_resample(X, y)
print(X.shape, y.shape)

(163992, 5) (163992,)


In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

In [65]:
knn = KNeighborsClassifier(n_neighbors=7, metric='minkowski', p=2).fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy_score(y_test, y_pred)

0.9082076507175089

In [66]:
confusion_matrix(y_test, y_pred)

array([[20335,  4264],
       [  252, 24347]], dtype=int64)

## Задание 3
* Самостоятельно реализуйте метрики Accuracy, Precision, Recall, F1
* Самостоятельно реализуйте kNN и Naive Bayes

## Дополнительные баллы

* Самостоятельно реализуйте SVM с возможностью выбирать ядро.